<a href="https://colab.research.google.com/github/PriyankaGona/MachineLearning-Assignments/blob/master/hw1_problem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
from keras.datasets import mnist
import numpy as np


Using TensorFlow backend.


In [0]:
# convert digits to binary values
def to_categorical(y,nb_classes):
  output = []
  for i in y:
    temp = np.zeros(nb_classes)
    temp[i] = 1
    output.append(temp)
  output = np.array(output)
  output = output.reshape(y.shape[0],nb_classes)
  return output

In [0]:
# intitalizing weights and bias to zero
def init(dim):
    W = np.zeros(shape=(dim, num_classes))
    b = 0
    return W, b

In [0]:
# sigmoid activation function
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [0]:
# MSE loss function
def mean_square_loss(A, Y):
    loss = np.mean((Y - A)**2)
    loss = np.squeeze(loss)
    return loss

In [0]:
# backward propagation
def backward_propagate(A, X, Y, batch_size):
    m = batch_size
    # gradient computation
    dW = (1 / m) * np.dot(X, ((A-Y) * A * (1-A)).T)
    db = (1 / m) * np.sum((A-Y) * A * (1-A))

    return dW, db

In [0]:
# forward porpagation
def forward_propogate(W, b, X):
    linear_transformation = np.dot(W.T, X) + b
    output = sigmoid(linear_transformation)
    return output

In [0]:
# mini-batch stochastic gradient descent
def sgd(W, b, X, Y, epochs, learning_rate):

    for j in range(epochs):
        training_loss = []
        for i in range(0, X.shape[1], batch_size):
            x_train_mini = X.T[i:i+batch_size]
            y_train_mini = Y.T[i:i+batch_size]
            output = forward_propogate(W,b,x_train_mini.T)
            loss = mean_square_loss(output,y_train_mini.T)
            dW, db = backward_propagate(output,x_train_mini.T,y_train_mini.T,batch_size)

            # gradient descent
            W = W - learning_rate * dW
            b = b - learning_rate * db

            training_loss.append(loss)
        epochLoss=sum(training_loss)/len(training_loss)
        print("Epoch {}/{}\t - loss : {}".format(j+1,epochs,round(epochLoss,4)))
    return W, b

In [0]:
# calculating accuracy of test data after training
def predict(W, b, X):

    m = X.shape[1]
    y_pred = np.zeros((1, m))
    W = W.reshape(X.shape[0], num_classes)
    A = forward_propogate(W,b,X)

    for i in range(A.shape[1]):
        y_pred[0, i] = 1 if A[0, i] > 0.5 else 0

    return y_pred

In [0]:
def model(X_train, Y_train, X_test, Y_test, epochs=50, learning_rate=0.1):

    y_train_pred = Y_train
    y_test_pred = Y_test

    # shape of predict_y_train and predict_y_test is 60000,10 similar to that of y_train and y_test categorical values
    y_train_pred = to_categorical(y_train_pred, 10)
    y_test_pred = to_categorical(y_test_pred, 10)

    test_accuracy_list=[]
    for i in range(0, 10):
        # set the classifier digit
        classifier = i
        print("\nClassifier: {}".format(classifier))
        print("=======================================")

        # modify training labels to create single class classification
        y_train_modified = np.array(Y_train)
        y_train_modified = np.where(y_train_modified == classifier, 1, 0)

        y_test_modified = np.array(Y_test)
        y_test_modified = np.where(y_test_modified == classifier, 1, 0)

        # core of the model
        W, b = init(X_train.shape[0])
        W, b = sgd(W, b, X_train, y_train_modified, epochs, learning_rate)

        # predicting the values based on trained weights and bias values
        Y_prediction_train = predict(W, b, X_train)
        Y_prediction_test = predict(W, b, X_test)

        # storing the values in the corresponding y_train labels
        y_train_pred[:, [i]] = Y_prediction_train.T
        y_test_pred[:, [i]] = Y_prediction_test.T

        # train and test classifier accuracy for each digit
        train_accuracy = round((100 - np.mean(np.abs(Y_prediction_train - y_train_modified)) * 100),4)
        test_accuracy = round((100 - np.mean(np.abs(Y_prediction_test - y_test_modified)) * 100),4)
        test_accuracy_list.append(test_accuracy)

        print("\nAccuracy of classifer {} in training : {} %".format(i, train_accuracy))
        print("Accuracy of classifer {} in testing : {} %".format(i, test_accuracy))
        print("\n")

    # coverting y_train values to binary values
    Y_train = to_categorical(Y_train, 10)
    Y_test = to_categorical(Y_test, 10)

    # overall test and train classifier accuracy of the network
    print("Overall accuracy in Training: {} %".format(round((100 - np.mean(np.abs(y_train_pred - Y_train)) * 100),4)))
    print("Overall accuracy in Testing: {} %".format(round((100 - np.mean(np.abs(y_test_pred - Y_test)) * 100),4)))
    print("Classifier with Strongest output is: ",np.argmax(test_accuracy_list))

In [0]:
# variables
batch_size = 32
num_classes = 1
epochs = 12

# Image Dimensions of MNIST
rows = 28
cols = 28

# loading MNIST dataset using keras
(x_train, y_train), (x_test, y_test) = mnist.load_data()


# reshaping dataset
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2]).T
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2]).T

# Normalization
x_train = x_train / 255
x_test = x_test / 255

y_train = y_train.T
y_test = y_test.T


model(x_train, y_train, x_test, y_test, epochs=12,learning_rate=0.01)

11493376/11490434 [==============================] - 0s 0us/step

Classifier: 0
Epoch 1/12	 - loss : 0.0518
Epoch 2/12	 - loss : 0.0255
Epoch 3/12	 - loss : 0.0209
Epoch 4/12	 - loss : 0.0185
Epoch 5/12	 - loss : 0.017
Epoch 6/12	 - loss : 0.0159
Epoch 7/12	 - loss : 0.0151
Epoch 8/12	 - loss : 0.0144
Epoch 9/12	 - loss : 0.0139
Epoch 10/12	 - loss : 0.0134
Epoch 11/12	 - loss : 0.013
Epoch 12/12	 - loss : 0.0127

Accuracy of classifer 0 in training : 98.7167 %
Accuracy of classifer 0 in testing : 98.96 %



Classifier: 1
Epoch 1/12	 - loss : 0.048
Epoch 2/12	 - loss : 0.0239
Epoch 3/12	 - loss : 0.0193
Epoch 4/12	 - loss : 0.0171
Epoch 5/12	 - loss : 0.0157
Epoch 6/12	 - loss : 0.0147
Epoch 7/12	 - loss : 0.014
Epoch 8/12	 - loss : 0.0134
Epoch 9/12	 - loss : 0.013
Epoch 10/12	 - loss : 0.0126
Epoch 11/12	 - loss : 0.0123
Epoch 12/12	 - loss : 0.012

Accuracy of classifer 1 in training : 98.7967 %
Accuracy of classifer 1 in testing : 99.05 %



Classifier: 2
Epoch 1/12	 - loss : 0.075